# 数据生成二进制文件

In [11]:
!rm ../data/IWSLT15/zh_en/model/prev_ctx/processed/*

In [12]:
!python preprocess.py --source-lang ti --target-lang zh\
--trainpref ../data/CCMT/bpe/train.bpe \
--validpref ../data/CCMT/bpe/dev.bpe \
--testpref ../data/CCMT/bpe/test.bpe \
--criterion=label_smoothed_cross_entropy \
--fp16 --workers 16 \
--destdir ../data/CCMT/model/baseline/processed
# --joined-dictionary \

2021-04-20 14:30:22 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='label_smoothed_cross_entropy', dataset_impl='mmap', destdir='../data/CCMT/model/baseline/processed', empty_cache_freq=0, fp16=True, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, simul_type=None, source_lang='ti', srcdict=None, suppress_crashes=False, target_lang='zh', task='translation', tensorboard_logdir=None, testpref='../data/CCMT/bpe/test.bpe', tgtdict=None, thres

# 训练模型

In [ ]:
!CUDA_VISIBLE_DEVICES=0 nohup python train.py ../data/CCMT/model/baseline/processed \
--save-dir='../data/CCMT/model/baseline/checkpoints' \
--source-lang=ti --target-lang=zh \
--arch=transformer \
--fp16 --memory-efficient-fp16 \
--num-workers 8 \
--max-tokens 10000 \
--left-pad-source \
--save-interval=1 --no-progress-bar --keep-last-epochs=5 --keep-best-checkpoints=5 \
--attention-dropout=0.3 --criterion=label_smoothed_cross_entropy \
--lr-scheduler=inverse_sqrt --stop-min-lr 1e-09 --label-smoothing=0.1 --update-freq=2 \
--optimizer adam --adam-betas '(0.9, 0.98)' --warmup-init-lr 1e-07 \
--lr 0.0005 --warmup-updates 8000 --dropout 0.3 --weight-decay 0.0001 \
--valid-subset=valid --max-epoch=100 \
--eval-bleu --eval-bleu-remove-bpe \
--skip-invalid-size-inputs-valid-test \
--best-checkpoint-metric 'ppl' \
--best-checkpoint-metric 'bleu' \
--maximize-best-checkpoint-metric \
--ddp-backend=no_c10d >../data/CCMT/model/baseline/log.txt  2>&1 &

In [9]:
!CUDA_VISIBLE_DEVICES=0 python train.py ../data/CCMT/model/backtrans/processed \
--save-dir='../data/CCMT/model/backtrans/checkpoints' \
--source-lang=zh --target-lang=ti \
--arch=transformer \
--fp16 --memory-efficient-fp16 \
--num-workers 8 \
--max-tokens 10000 \
--left-pad-source \
--save-interval=1 --no-progress-bar --keep-last-epochs=5 --keep-best-checkpoints=5 \
--attention-dropout=0.3 --criterion=label_smoothed_cross_entropy \
--lr-scheduler=inverse_sqrt --stop-min-lr 1e-09 --label-smoothing=0.1 --update-freq=2 \
--optimizer adam --adam-betas '(0.9, 0.98)' --warmup-init-lr 1e-07 \
--lr 0.0005 --warmup-updates 8000 --dropout 0.3 --weight-decay 0.0001 \
--valid-subset=valid --max-epoch=100 \
--eval-bleu --eval-bleu-remove-bpe \
--skip-invalid-size-inputs-valid-test \
--best-checkpoint-metric 'ppl' \
--best-checkpoint-metric 'bleu' \
--maximize-best-checkpoint-metric \
--ddp-backend=no_c10d

2021-04-20 13:43:23 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distribute

2021-04-20 13:43:23 | INFO | fairseq.tasks.translation | [zh] dictionary: 17976 types
2021-04-20 13:43:23 | INFO | fairseq.tasks.translation | [ti] dictionary: 7696 types
2021-04-20 13:43:23 | INFO | fairseq.data.data_utils | loaded 729 examples from: ../data/CCMT/model/backtrans/processed/valid.zh-ti.zh
2021-04-20 13:43:23 | INFO | fairseq.data.data_utils | loaded 729 examples from: ../data/CCMT/model/backtrans/processed/valid.zh-ti.ti
2021-04-20 13:43:23 | INFO | fairseq.tasks.translation | ../data/CCMT/model/backtrans/processed valid zh-ti 729 examples
2021-04-20 13:43:23 | INFO | fairseq_cli.train | TransformerModel(
  (encoder): TransformerEncoder(
    (dropout_module): FairseqDropout()
    (embed_tokens): Embedding(17976, 512, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (dropout_module): FairseqDropout()
          (k_proj): Linear(in_features=

2021-04-20 13:43:31 | INFO | fairseq.utils | ***********************CUDA enviroments for all 1 workers***********************
2021-04-20 13:43:31 | INFO | fairseq.utils | rank   0: capabilities =  7.5  ; total memory = 47.461 GB ; name = Quadro RTX 8000                         
2021-04-20 13:43:31 | INFO | fairseq.utils | ***********************CUDA enviroments for all 1 workers***********************
2021-04-20 13:43:31 | INFO | fairseq_cli.train | training on 1 devices (GPUs/TPUs)
2021-04-20 13:43:31 | INFO | fairseq_cli.train | max tokens per GPU = 10000 and batch size per GPU = None
2021-04-20 13:43:31 | INFO | fairseq.trainer | Preparing to load checkpoint ../data/CCMT/model/backtrans/checkpoints/checkpoint_last.pt
2021-04-20 13:43:31 | INFO | fairseq.trainer | No existing checkpoint found ../data/CCMT/model/backtrans/checkpoints/checkpoint_last.pt
2021-04-20 13:43:31 | INFO | fairseq.trainer | loading train data for epoch 1
2021-04-20 13:43:31 | INFO | fairseq.data.data_utils | l

# 翻译

In [15]:
!CUDA_VISIBLE_DEVICES=0 python generate.py ../data/CCMT/model/baseline/processed \
--device-id 6 \
--path ../data/CCMT/model/baseline/checkpoints/checkpoint_best.pt \
--batch-size 128 --beam 4 --lenpen 0.6 \
--criterion=label_smoothed_cross_entropy \
--source-lang=ti --target-lang=zh \
--skip-invalid-size-inputs-valid-test \
--remove-bpe \
# --results-path ../data/IWSLT15/zh_en/model/auto_ctx
# --max-len-b 1000 \
# --dataset-impl raw \

2021-04-20 18:47:05 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False}, 'common_eval': {'_name': None, 'path': '../data/CCMT/model/baseline/checkpoints/checkpoint_best.pt', 'post_process': 'subword_nmt', 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size

S-817	འཛིན ་ བཟུང ་ བཅའ ་ ཁྲིམས ་
T-817	捕亡律
H-817	-0.568135678768158	法律 、
D-817	-0.568135678768158	法律 、
P-817	-0.8502 -0.1327 -0.1154
S-453	དེ ་ རྗེས ་ ཤིགཆོག ་ གམ
T-453	请 把 它 换 了 好 吗
H-453	-1.5630757808685303	以后 可以 吗 ?
D-453	-1.5630757808685303	以后 可以 吗 ?
P-453	-0.5798 -1.3219 -1.9615 -0.0664 -0.1759
S-231	མེ ་ འཁོར ་ བསྐོར ་ བ །
T-231	列车 一开 就 开放 。
H-231	-0.8294795155525208	火车 、
D-231	-0.8294795155525208	火车 、
P-231	-1.2428 -0.2500 -0.1108
S-727	ཤིས ་ ཨན ་ བར ་ དུ །
T-727	一直 到 西安 ,
H-727	-1.0002988576889038	安妮 ,
D-727	-1.0002988576889038	安妮 ,
P-727	-0.4590 -1.4252 -0.3105 -0.1034
S-691	ཁྱོས ་ སེམས ་ མ ་ སྡུག །
T-691	您 别难过 了 。
H-691	-0.8845407962799072	你 不要 伤心
D-691	-0.8845407962799072	你 不要 伤心
P-691	-1.0103 -0.1238 -0.7283 -0.1697
S-796	བཅིངས ་ འགྲོལ ་ ཐོབ ་ ཚུན །
T-796	解放以来 。
H-796	-0.1289205104112625	解放以来 。
D-796	-0.1289205104112625	解放以来 。
P-796	-0.0601 -0.0137 -0.0964 -0.1260
S-437	ཆུ ་ འདྲེན ་ རྒྱུ ་ མེད །
T-437	没有 水 抽 了 ,
H-437	-2.2340545654296875	灌溉 不 安全 。
D-437	-2.2340545654296875

S-96	ལེའུ ་ བརྒྱད ་ པ །
T-96	第八章 ,
H-96	-0.08889108151197433	第八章
D-96	-0.08889108151197433	第八章
P-96	-0.0175 -0.0063 -0.1481
S-290	ཞིན ་ ཧྥུའུ ་ མེན །
T-290	<<unk>> 福敏
H-290	-1.9665868282318115	新福尼亚 、
D-290	-1.9665868282318115	新福尼亚 、
P-290	-0.0607 -1.1932 -3.7285 -0.0757 -0.1073
S-378	ངའི ་ བློར ་ འབབ །
T-378	我 满意 。
H-378	-1.0885024070739746	我 想 ,
D-378	-1.0885024070739746	我 想 ,
P-378	-0.1210 -1.8956 -0.3749 -0.1092
S-761	བཟོ ་ ལས ་ ཐད །
T-761	工业 方面 ,
H-761	-0.27643802762031555	在 工业 方面 ,
D-761	-0.27643802762031555	在 工业 方面 ,
P-761	-0.3333 -0.0061 -0.0060 -0.2551 -0.1256
S-132	འགྲིག ་ ཡོད ་ དམ །
T-132	好了吗 ?
H-132	-0.9448887705802917	行 吗 ?
D-132	-0.9448887705802917	行 吗 ?
P-132	-1.8364 -0.0902 -0.1164 -0.1278
S-451	ཝུའུ ་ ཀྲུང ་ ཙེ །
T-451	吴忠泽 ,
H-451	-2.056628942489624	毛泽东 同志 ,
D-451	-2.056628942489624	毛泽东 同志 ,
P-451	-3.2913 -1.1321 -0.1777 -0.1238
S-6	ངས ་ བལྟས ་ ན །
T-6	要 我 看 呢 !
H-6	-0.7344660758972168	我 想 ,
D-6	-0.7344660758972168	我 想 ,
P-6	-1.3455 -0.0255 -0.1909 -0.1255
S-658	ཟླ 3 ཚེ

H-47	-0.7862677574157715	扎实 推进 。
D-47	-0.7862677574157715	扎实 推进 。
P-47	-0.8395 -0.1096 -0.7292 -0.1281
S-40	སྲིད ་ གཞུང ་ གི ་ ལས ་ དོན ་ ནང
T-40	政府 工作 中
H-40	-0.56049644947052	在 政府 工作 中 ,
D-40	-0.56049644947052	在 政府 工作 中 ,
P-40	-0.9100 -0.1064 -0.1762 -0.2082 -0.1184 -0.1231
S-36	ལས ་ བགོས ་ འགན ་ ཁུར ་ དང ་ །
T-36	分工负责 ,
H-36	-0.13491889834403992	分工负责 ,
D-36	-0.13491889834403992	分工负责 ,
P-36	-0.0406 -0.0022 -0.1500 -0.1172
S-913	དོ ་ དམ ་ སྲོལ ་ ཡིག ་ ཆོས ་ ལུགས
T-913	管理条例 宗教
H-913	-0.7562383413314819	管理 规章
D-913	-0.7562383413314819	管理 规章
P-913	-0.5879 -0.5271 -0.3469
S-889	རང ་ བྱུང ་ གནོད ་ འཚེ ་ ཆེན ་ པོ
T-889	大 自然灾害
H-889	-1.3882274627685547	自然灾害
D-889	-1.3882274627685547	自然灾害
P-889	-1.2184 -0.8857
S-708	མི ་ གྲངས ་ ཉུང ་ དུ ་ གཏོང ་ བ
T-708	减少 数量
H-708	-0.21538279950618744	减少 数量 ,
D-708	-0.21538279950618744	减少 数量 ,
P-708	-0.0274 -0.2495 -0.0936 -0.1244
S-548	སྤྱི ་ ཚོགས ་ རིང ་ ལུགས ་ དཔལ ་ འབྱོར
T-548	社会主义 经济
H-548	-0.9361358880996704	社会主义 经济 、
D-548	-0.9361358880996704	社会主义 经济 

P-886	-0.0368 -0.0251 -0.0094 -0.1209
S-885	འགན ་ སྲུང ་ གི ་ གནད ་ དོན
T-885	保障 问题
H-885	-0.2124873548746109	保障 问题 ,
D-885	-0.2124873548746109	保障 问题 ,
P-885	-0.0645 -0.1106 -0.1946 -0.1184
S-196	ཕལ ་ ཆེར ་ གཟའ ་ འཁོར ་ གཅིག
T-196	大约 一周 。
H-196	-1.051889419555664	大约 一个 星期
D-196	-1.051889419555664	大约 一个 星期
P-196	-0.9442 -0.0122 -0.1072 -1.3529
S-219	ཁོང ་ ཚོའི ་ ལས ་ ཀ ་ དང
T-219	他们 解决 工作
H-219	-1.0193297863006592	积极 为 他们 的 工作 、
D-219	-1.0193297863006592	积极 为 他们 的 工作 、
P-219	-0.9797 -0.8464 -0.0306 -0.6683 -0.0312 -0.6026 -0.1175
S-309	ཡིག ་ འབྲུ ་ དྲུག ་ པོ ་ འདི
T-309	这 六个 字
H-309	-0.13384802639484406	这 六个 字 ,
D-309	-0.13384802639484406	这 六个 字 ,
P-309	-0.0099 -0.1148 -0.0324 -0.0795 -0.1149
S-310	དགྲ ་ བོས ་ མགོ ་ བཏགས ་ པ
T-310	敌人 已经 投降
H-310	-0.09884438663721085	敌人 已经 投降 。
D-310	-0.09884438663721085	敌人 已经 投降 。
P-310	-0.0107 -0.0085 -0.0126 -0.1020 -0.1257
S-442	ལག ་ དངུལ ་ སྦྱིན ་ རོགས ་ གནང
T-442	请 付现金
H-442	-2.4933338165283203	助金 。
D-442	-2.4933338165283203	助金 。
P-442	-4.1933 -1.0

...

H-499	-0.9491696357727051	任何 单位 不得 增加 条件 。
D-499	-0.9491696357727051	任何 单位 不得 增加 条件 。
P-499	-0.0044 -0.0751 -0.1634 -2.5686 -0.0065 -0.1028 -0.1299
S-621	སྤྱི ་ ཚོགས ་ རིང ་ ལུགས ་ ཀྱི ་ མི ་ རིགས ་ འབྲེལ ་ བ ་ སྲ ་ བརྟན ་ དང ་ འཕེལ ་ རྒྱས
T-621	巩固 和 发展 社会主义 的 民族 关系
H-621	-0.2448926717042923	巩固 和 发展 社会主义 民族 关系 ,
D-621	-0.2448926717042923	巩固 和 发展 社会主义 民族 关系 ,
P-621	-0.0078 -0.1002 -0.0586 -0.1262 -0.1600 -0.1296 -0.1418 -0.1288
S-176	ངས ་ ནད ་ བྱུང ་ ནས ་ སྨན ་ ཁང ་ དུ ་ སྡོད ་ པའི ་ ཏུང ་ ཏུང ་ ལ ་ སྟོན ་ རྒྱུ
T-176	我 给 生病 住院 的 东东 看
H-176	-2.3455240726470947	我病 了 , 我要 去 住院 。
D-176	-2.3455240726470947	我病 了 , 我要 去 住院 。
P-176	-1.5317 -0.9004 -0.6426 -0.0826 -1.3953 -1.1840 -2.7454 -0.1593 -0.1244
S-28	དེ ་ ནི ་ རྒྱལ ་ ཁབ ་ ཕྱི ་ ནང ་ གི ་ རྙོག ་ འཛིང ་ ཆེ ་ བའི ་ དུས ་ བབ ་ འོག
T-28	这 是因为 要 在 复杂 的 国内外 形势 下
H-28	-0.6520199179649353	这 是因为 , 要 在 复杂 的 国内外 形势 下 ,
D-28	-0.6520199179649353	这 是因为 , 要 在 复杂 的 国内外 形势 下 ,
P-28	-0.8883 -0.0983 -0.1265 -0.3542 -0.8176 -0.1669 -0.1155 -0.0081 -0.0773 -0

S-560	སྲུང ་ འཁོར ་ འདིས ་ ཁྱོད ་ ལ ་ ས ་ གཞིའི ་ སྲོག ་ རྩ ་ འཚོལ ་ རོགས ་ བྱེད ་ ངེས
T-560	这个 护身符 会带 你 找到 大地 之脉
H-560	-3.713545083999634	这位 警卫 也 可能 帮助 你 寻找 根据地 。
D-560	-3.713545083999634	这位 警卫 也 可能 帮助 你 寻找 根据地 。
P-560	-2.4901 -1.8285 -0.1694 -3.1305 -0.4761 -0.9759 -0.1095 -1.5534 -4.7107 -0.0831 -0.1269
S-184	སྐྱར ་ མོ ་ འཚོ ་ བའི ་ ཀུའུ ་ རོང ་ གླིང ་ ཕྲན ་ དུ ་ གཏན ་ ནས ་ མི ་ འགྲོ
T-184	从来 都 不 去 鹭鸟 呆 的 古<<unk>> 岛
H-184	-4.014729022979736	为 生活 在 吉林格拉西拉 群岛 去 。
D-184	-4.014729022979736	为 生活 在 吉林格拉西拉 群岛 去 。
P-184	-1.6342 -2.4890 -0.1052 -2.0224 -1.4659 -1.6853 -1.0553 -3.0388 -0.3089 -3.2354 -0.8553 -0.6857 -0.1262
 78%|███████████████████████████▏       | 7/9 [00:04<00:01,  1.37it/s, wps=1308]S-50	འདི ་ ན ་ བཤད ་ པའི ་ ཞེན ་ པ ་ དེ ་ ནི ་ དགའ ་ ཞེན ་ སྣ ་ ཚོགས ་ ཀྱི ་ འཛོམ ་ པའི ་ ལས ་ ཀྱི ་ བརྩེ ་ ཞེན ་ དེ ་ ཡིན །
T-50	此处 谈到 的 爱 是 指 由 诸般 的 爱 汇集 而生 的 工作 之爱 。
H-50	-3.491279125213623	这里 所说 的 这个 爱 就是 喜悦 所 爱 的 感情 。
D-50	-3.491279125213623	这里 所说 的 这个 爱 就是 喜悦 所 爱 的 感情 。
P-50	-2.8266 -2.00

S-769	སྲན ་ ཕྱུར ་ ཐེངས ་ བཞི ་ ལྔ ་ ཙམ ་ ལ ་ ཚྭ ་ སྦྱངས ་ བྱས ་ རྗེས ་ ད ་ གཟོད ་ ཡག ་ པོ ་ འཁྱོངས ་ པས ་ བཙོང ་ ཆོག
T-769	豆腐 要 <<unk>> 四五次 才 能够 卖 才 <<unk>> 的 好
H-769	-3.1250202655792236	只有 五四 五次 五分钟 才 盐粮 了 。
D-769	-3.1250202655792236	只有 五四 五次 五分钟 才 盐粮 了 。
P-769	-0.3708 -1.1820 -1.1906 -0.8536 -0.9983 -1.7733 -2.4721 -0.1388 -3.1038 -1.5543 -0.3492 -0.4438 -0.1315
S-565	ལ ་ ཏིང ་ མེ ་ གླིང ་ འཚམས ་ འདྲི ་ གནང ་ རིང ་ ཆི ་ ལིའི ་ སན ་ ཐེ ་ ཡ ་ ཀོ ་ ནས ་ མཉམ ་ འབྲེལ ་ རྒྱལ ་ ཚོགས
T-565	访问 拉丁美洲 期间 在 智利 圣地亚哥 联合国
H-565	-2.594303846359253	在 拉丁美洲 期间 在 智利 圣地亚哥 联合国 拉丁美洲 ,
D-565	-2.594303846359253	在 拉丁美洲 期间 在 智利 圣地亚哥 联合国 拉丁美洲 ,
P-565	-1.4356 -0.6020 -0.0513 -0.2191 -0.9807 -0.1354 -0.1330 -3.4329 -0.3251 -1.0450 -1.5227 -0.1409 -2.8323 -0.2362 -0.4728 -0.1279
S-57	དབང ་ པོ ་ སྐྱོན ་ ཤོར ་ རིགས ་ སྣ ་ ཚོགས ་ ཀྱི ་ ཁྱད ་ ཆོས ་ དང ་ དགོས ་ མཁོ ་ མི ་ འདྲ ་ བར ་ བསྟུན ་ ནས ་
T-57	适应 各类 残疾人 的 不同 特点 和 需要 ,
H-57	-0.21655789017677307	适应 各类 残疾人 的 不同 特点 和 需要 ,
D-57	-0.21655789017677307	适应 各类 残疾人 的 不同 特点 

S-114	སྒོ ་ བརྒྱབ ་ ནས ་ འཛུགས ་ སྐྲུན ་ བྱས ་ ན ་ ལེགས ་ འབྲས ་ འཐོབ ་ མི ་ ཐུབ ་ པ ་ གསལ ་ པོར ་ མཚོན ་ ཡོད །
T-114	关起门来 搞 建设 是 不能 成功 的 。
H-114	-2.062300205230713	关起门来 搞 建设 是 得不到 的 。
D-114	-2.062300205230713	关起门来 搞 建设 是 得不到 的 。
P-114	-1.9794 -0.0035 -0.0380 -0.0021 -0.7892 -0.0825 -2.2173 -2.3225 -1.0856 -0.0371 -0.1361
S-470	ཉིན ་ ལྟར ་ གཞན ་ གྱི ་ རྒྱུ ་ ནོར ་ ཧམ ་ སེམས ་ བྱས ་ ནས ་ ཉེས ་ པ ་ ཕོག ་ མཁན ་ མཐོང ་ རྒྱུ ་ ཡོད །
T-470	每天 都 可以 看到 因 贪图 别人 的 财产 而 受到 惩罚 的 人 ,
H-470	-0.14477533102035522	每天 都 可以 看到 因 贪图 别人 的 财产 而 受到 惩罚 的 人 ,
D-470	-0.14477533102035522	每天 都 可以 看到 因 贪图 别人 的 财产 而 受到 惩罚 的 人 ,
P-470	-0.1261 -0.0568 -0.0164 -0.0097 -0.0174 -0.0345 -0.0491 -0.0242 -0.0617 -0.0159 -0.0038 -0.0050 -0.0587 -0.0829 -0.0254 -0.0839 -0.1209
S-477	མི ་ མི ་ འདྲ ་ བ ་ དང ་ ལས ་ རིགས ་ མི ་ འདྲ ་ བར ་ སྐད ་ ཆའི ་ བཤད ་ སྟངས ་ ཡང ་ མི ་ འདྲ །
T-477	就是说 不同 的 人 啊 , 不同 的 职业 , 不同 的 风格 不 一样 。
H-477	-1.970366358757019	不同 的 语言 也 不同 , 不同 , 不同 种类 也 不同 。
D-477	-1.970366358757019	不同 的 语言 也 不同 , 不同 , 不同 

 89%|███████████████████████████████    | 8/9 [00:05<00:00,  1.28it/s, wps=1287]S-482	སྐབས ་ དེར ་ ངས ་ ངས ་ ངས ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་ ་
T-482	当时 我 唱 了 一个
H-482	-7.047879695892334	那时 我 对 我 说 , 我 给 你 帮 我 按 我 做 的 跟 我 分名作 的 ,
D-482	-7.047879695892334	那时 我 对 我 说 , 我 给 你 帮 我 按 我 做 的 跟 我 分名作 的 ,
P-482	-1.9068 -0.3136 -4.5192 -0.9278 -0.7620 -3.3647 -1.4965 -1.1170 -2.3743 -3.0125 -0.1217 -4.4732 -0.2941 -3.5428 -1.4506 -1.3073 -2.0179 -1.9816 -3.2115 -3.8598 -2.2756 -1.6544 -0.2635
2021-04-20 18:47:12 | INFO | fairseq_cli.generate | NOTE: hypothesis and token scores are output in base 2
2021-04-20 18:47:12 | INFO | fairseq_cli.generate | Translated 985 sentences (7,287 tokens) in 2.4s (415.03 sentences/s, 3070.41 tokens/s)
Generate test with beam=4: BLEU4 = 34.80, 51.0/35.2/29.7/27.5 (BP=1.000, ratio=1.057, syslen=5637, reflen=5334)


# Tools

In [1]:
!ls ../data/IWSLT15/zh_en/model/prev_ctx/checkpoints

checkpoint100.pt  checkpoint99.pt		checkpoint.best_ppl_19.33.pt
checkpoint96.pt   checkpoint.best_ppl_18.70.pt	checkpoint.best_ppl_19.41.pt
checkpoint97.pt   checkpoint.best_ppl_18.76.pt	checkpoint_best.pt
checkpoint98.pt   checkpoint.best_ppl_18.92.pt	checkpoint_last.pt


In [2]:
!nvidia-smi

Sun Dec 13 08:41:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.43       Driver Version: 418.43       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 47%   68C    P2   200W / 250W |   9902MiB / 11178MiB |     92%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 21%   36C    P8     9W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
!kill 11049 11050 11051